# Simple YOLO Label Checker

A straightforward notebook to view YOLO labels without complex widgets.

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Configuration
INPUT_DIR = "/Users/marcelmerkx/Development/universal-scanner/data/OCR/cutouts"
OUTPUT_DIR = "/Users/marcelmerkx/Development/universal-scanner/data/OCR/labels"

In [ ]:
# Helper functions
def read_yolo_labels(label_path):
    """Read YOLO format labels from file"""
    boxes = []
    
    try:
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    boxes.append({
                        'class_id': class_id,
                        'x_center': x_center,
                        'y_center': y_center,
                        'width': width,
                        'height': height
                    })
    except Exception as e:
        print(f"Error reading label file {label_path}: {e}")
    
    return boxes

def yolo_to_xyxy(box, img_width, img_height):
    """Convert YOLO format to xyxy format"""
    x_center = box['x_center'] * img_width
    y_center = box['y_center'] * img_height
    width = box['width'] * img_width
    height = box['height'] * img_height
    
    x1 = x_center - width / 2
    y1 = y_center - height / 2
    x2 = x_center + width / 2
    y2 = y_center + height / 2
    
    return x1, y1, x2, y2

def get_image_label_pairs():
    """Get all image files that have corresponding label files"""
    images_dir = Path(INPUT_DIR)
    labels_dir = Path(OUTPUT_DIR)
    
    extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    pairs = []
    
    # Get all label files
    label_files = list(labels_dir.glob("*.txt"))
    
    for label_file in sorted(label_files):
        # Look for corresponding image
        base_name = label_file.stem
        image_file = None
        
        for ext in extensions:
            potential_image = images_dir / f"{base_name}{ext}"
            if potential_image.exists():
                image_file = potential_image
                break
            # Try uppercase extension
            potential_image = images_dir / f"{base_name}{ext.upper()}"
            if potential_image.exists():
                image_file = potential_image
                break
        
        if image_file:
            pairs.append((image_file, label_file))
    
    return pairs

In [ ]:
# Get all labeled images
pairs = get_image_label_pairs()
print(f"Found {len(pairs)} labeled images")

In [ ]:
# Function to display a single image with labels
def show_labeled_image(index):
    if index < 0 or index >= len(pairs):
        print(f"Invalid index. Please use 0-{len(pairs)-1}")
        return
        
    image_path, label_path = pairs[index]
    
    # Read image
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Error reading image: {image_path}")
        return
        
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]
    
    # Read labels
    boxes = read_yolo_labels(label_path)
    
    # Create figure
    plt.figure(figsize=(12, 8))
    plt.imshow(image_rgb)
    
    # Draw boxes
    colors = plt.cm.rainbow(np.linspace(0, 1, max(1, len(boxes))))
    
    for i, (box, color) in enumerate(zip(boxes, colors)):
        x1, y1, x2, y2 = yolo_to_xyxy(box, w, h)
        
        # Create rectangle
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor='none'
        )
        plt.gca().add_patch(rect)
        
        # Add box number
        plt.text(
            x1, y1 - 5, f'Box {i+1}',
            color=color, fontsize=10, weight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7)
        )
    
    plt.title(f"Image: {image_path.name} | Boxes: {len(boxes)} | Index: {index + 1}/{len(pairs)}")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## View Individual Images

Change the index in the cell below to view different images (0 to 9 for your 10 labeled images):

In [ ]:
# View image by index (change this number to see different images)
show_labeled_image(0)

In [ ]:
# View next image
show_labeled_image(1)

In [ ]:
# View another image
show_labeled_image(2)

## View All Images in a Grid

In [ ]:
# Show all labeled images in a grid
n_images = len(pairs)
cols = 3
rows = (n_images + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(18, 6 * rows))
if rows == 1:
    axes = axes.reshape(1, -1)
if n_images == 1:
    axes = np.array([[axes]])

for idx, (image_path, label_path) in enumerate(pairs):
    row = idx // cols
    col = idx % cols
    
    # Read image
    image = cv2.imread(str(image_path))
    if image is not None:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:2]
        
        # Read labels
        boxes = read_yolo_labels(label_path)
        
        # Display image
        axes[row, col].imshow(image_rgb)
        
        # Draw boxes
        colors = plt.cm.rainbow(np.linspace(0, 1, max(1, len(boxes))))
        for i, (box, color) in enumerate(zip(boxes, colors)):
            x1, y1, x2, y2 = yolo_to_xyxy(box, w, h)
            rect = patches.Rectangle(
                (x1, y1), x2 - x1, y2 - y1,
                linewidth=2, edgecolor=color, facecolor='none'
            )
            axes[row, col].add_patch(rect)
        
        axes[row, col].set_title(f"{image_path.name}\n{len(boxes)} boxes", fontsize=10)
        axes[row, col].axis('off')

# Hide empty subplots
for idx in range(n_images, rows * cols):
    row = idx // cols
    col = idx % cols
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

## Statistics

In [ ]:
# Analyze label statistics
box_counts = []
for _, label_path in pairs:
    boxes = read_yolo_labels(label_path)
    box_counts.append(len(boxes))

print(f"Total labeled images: {len(pairs)}")
print(f"Total boxes: {sum(box_counts)}")
print(f"Average boxes per image: {np.mean(box_counts):.2f}")
print(f"Min boxes: {min(box_counts)}")
print(f"Max boxes: {max(box_counts)}")

# Plot distribution
plt.figure(figsize=(8, 5))
plt.hist(box_counts, bins=range(0, max(box_counts) + 2), edgecolor='black')
plt.xlabel('Number of Boxes')
plt.ylabel('Image Count')
plt.title('Distribution of Box Counts per Image')
plt.grid(True, alpha=0.3)
plt.show()